In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [10]:
BATCH_SIZE = 100

INPUT_NODE = 784
OUTPUT_NODE = 10

LAYER1_NODE = 500

LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99

REGULARIZATION_RATE = 0.0001
TRAING_STEP = 3000
MOVING_AVERAGE_DECAY = 0.99

In [56]:
def inference(input_tensor, avg_class, W1, b1, W2, b2):
    if (avg_class == None):
        layer1 = tf.nn.relu(tf.matmul(input_tensor, W1) + b1)
        return tf.matmul(layer1, W2) + b2
    else:
        layer1 = tf.nn.relu(tf.matmul(input_tensor, avg_class.average(W1)) + avg_class.average(b1))
        return tf.matmul(layer1, avg_class.average(W2)) + avg_class.average(b2)

In [69]:
def train(mnist):
    x = tf.placeholder(tf.float32, shape=[None, INPUT_NODE], name="X-input")
    y = tf.placeholder(tf.float32, shape=[None, OUTPUT_NODE], name="Y-input")
    
    #第一層隱藏層
    W1 = tf.Variable(tf.random_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
    b1 = tf.Variable(tf.zeros(shape=[LAYER1_NODE]))
    #輸出層
    W2 = tf.Variable(tf.random_normal([LAYER1_NODE, OUTPUT_NODE], stddev=0.1))
    b2 =  tf.Variable(tf.zeros(shape=[OUTPUT_NODE]))
    
    y_ = inference(x, None, W1, b1, W2, b2)
    
    #定義存儲訓練輪數的變量
    global_step = tf.Variable(0, trainable=False)
    
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    
    variable_averages_op = variable_averages.apply(tf.trainable_variables())
    
    average_y = inference(x, variable_averages, W1, b1, W2, b2)
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_ , 
                                                                   labels=tf.argmax(y, 1))
    #計算當前batch中所有cross_entropy平均值
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    regulatizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    regularization = regulatizer(W1) + regulatizer(W2)
    
    loss = cross_entropy_mean + regularization
    
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE, 
                                               global_step, 
                                               mnist.train.num_examples / BATCH_SIZE, 
                                               LEARNING_RATE_DECAY)
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    #先不執行optimizer先執行variable_averages_op
    with tf.control_dependencies([optimizer, variable_averages_op]):
        optimizer = tf.no_op(name="train")
        
    correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        
        #驗證集
        validate_feed = {x: mnist.validation.images, y: mnist.validation.labels}
        test_feed = {x: mnist.test.images, y: mnist.test.labels}
        
        for i in range(6000):
            
            if (i % 1000 == 0):
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print("After %d training step(s), validation accuracy using avarage model is %g" % (i, validate_acc))
                
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(optimizer, feed_dict={x :xs, y:ys})
            
        test_acc = sess.run(accuracy, feed_dict=test_feed)
        print("After %d training step(s), test accuracy using avarage model is %g" % (30000, test_acc))
        
def main(argv=None):
#     mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
    train(mnist)

In [70]:
if __name__ == '__main__':
    tf.app.run()

After 0 training step(s), validation accuracy using avarage model is 0.1202
After 1000 training step(s), validation accuracy using avarage model is 0.9766
After 2000 training step(s), validation accuracy using avarage model is 0.9824
After 3000 training step(s), validation accuracy using avarage model is 0.9834
After 4000 training step(s), validation accuracy using avarage model is 0.9846
After 5000 training step(s), validation accuracy using avarage model is 0.986
After 30000 training step(s), test accuracy using avarage model is 0.9827


SystemExit: 

C:\Users\Philip\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
